In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall

import nest_asyncio

nest_asyncio.apply()

In [24]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [25]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [26]:
class MyOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-4o"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(
        self, tool_call: ChatCompletionMessageToolCall
    ) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

In [27]:
agent = MyOpenAIAgent(tools=[multiply_tool, add_tool])

In [28]:
agent.chat("Hi")

Trace ID is not set. Creating generation client with new trace id.


'Hello! How can I assist you today?'

In [29]:
agent.chat("What is 2123 * 215123")

Trace ID is not set. Creating generation client with new trace id.
Trace ID is not set. Creating generation client with new trace id.
Trace ID is not set. Creating generation client with new trace id.


'The result of \\( 2123 \\times 215123 \\) is 456,706,129.'

### Basics of querying using vector db

In [30]:
!ls -l docs/

total 10248
-rw-r--r--  1 arunesh  staff  4963538 Nov 12 17:04 paper_1.pdf


In [31]:
# Import necessary libraries
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from langfuse.llama_index import LlamaIndexInstrumentor

instrumentor = LlamaIndexInstrumentor()
 
# Automatically trace all LlamaIndex operations
instrumentor.start()

# Load environment variables
load_dotenv()

# Load documents from a directory (you can change this path as needed)
documents = SimpleDirectoryReader("docs/").load_data()

# Create an index from the documents
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine()

# Example query
response = query_engine.query("What is an SWE-agent?")

print(response)

 
# Flush events to langfuse
instrumentor.flush()

Trace ID is not set. Creating generation client with new trace id.
Trace ID is not set. Creating generation client with new trace id.
Trace ID is not set. Creating generation client with new trace id.


An SWE-agent is a system that facilitates LM agents to autonomously use computers to solve software engineering tasks. It includes a custom agent-computer interface (ACI) that significantly enhances an agent's ability to create and edit code files, navigate entire repositories, and execute tests and other programs.


In [21]:
from llama_index.embeddings.openai import OpenAIEmbedding
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize the OpenAI embedding model
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Define the trivia questions and matching answers
phrases = [
    "Who was the first president of the United States?",
    "What is the capital city of France?",
    "In what year did humans first land on the moon?",
    "Which element on the periodic table has the chemical symbol O?",
    "What is the largest planet in the solar system?",
    "The first president of the United States was George Washington.",
    "The capital city of France is Paris.",
    "Humans first landed on the moon in the year 1969.",
    "The chemical symbol O represents the element Oxygen.",
    "The largest planet in the solar system is Jupiter."
]

# Generate embeddings for each phrase using OpenAI embeddings
embeddings = embedding_model.get_text_embedding_batch(phrases)

# Convert embeddings to a numpy array
embeddings_array = np.array(embeddings)

# Print the first phrase and the first several elements of its embedding
print(f"Phrase: {phrases[0]}")
print(f"First 5 elements of its embedding: {embeddings_array[0][:5]}\n")

# Compute cosine similarity between the embeddings
similarity_matrix = cosine_similarity(embeddings_array)

# Print the cosine similarity matrix
print("Cosine Similarity Matrix:")
print(np.round(similarity_matrix, 2))
print("\nDetailed Similarity Results:\n")

# Output comparison between phrases with improved readability
for i in range(len(phrases)):
    for j in range(i + 1, len(phrases)):
        print(f"Cosine similarity between:\n  '{phrases[i]}'\n  and\n  '{phrases[j]}'\n  => {similarity_matrix[i, j]:.4f}\n")

Trace ID is not set. Creating generation client with new trace id.


Phrase: Who was the first president of the United States?
First 5 elements of its embedding: [-0.005321   -0.0220063  -0.01976715 -0.02281288 -0.00795742]

Cosine Similarity Matrix:
[[1.   0.78 0.83 0.76 0.76 0.92 0.75 0.78 0.72 0.73]
 [0.78 1.   0.74 0.75 0.77 0.75 0.94 0.72 0.73 0.73]
 [0.83 0.74 1.   0.75 0.78 0.78 0.72 0.93 0.72 0.74]
 [0.76 0.75 0.75 1.   0.77 0.73 0.74 0.73 0.93 0.75]
 [0.76 0.77 0.78 0.77 1.   0.72 0.75 0.75 0.73 0.93]
 [0.92 0.75 0.78 0.73 0.72 1.   0.79 0.78 0.74 0.76]
 [0.75 0.94 0.72 0.74 0.75 0.79 1.   0.74 0.76 0.78]
 [0.78 0.72 0.93 0.73 0.75 0.78 0.74 1.   0.74 0.76]
 [0.72 0.73 0.72 0.93 0.73 0.74 0.76 0.74 1.   0.76]
 [0.73 0.73 0.74 0.75 0.93 0.76 0.78 0.76 0.76 1.  ]]

Detailed Similarity Results:

Cosine similarity between:
  'Who was the first president of the United States?'
  and
  'What is the capital city of France?'
  => 0.7790

Cosine similarity between:
  'Who was the first president of the United States?'
  and
  'In what year did humans fi